scrape_the_salsePerson_name


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

# Set up Chrome WebDriver with Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(options=options)

# Navigate to the desired Amazon product page
product_url = "https://www.amazon.it/dp/B0B9HS2YCW"  # Replace with the desired product URL
driver.get(product_url)

# Extract the page source after the page is loaded
html = driver.page_source

# Close the Selenium WebDriver
driver.quit()

# Parse the page source using BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Find the seller name from the Buy Box
seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

if seller_element is not None:
    seller_name = seller_element.get_text(strip=True)
    print("Seller Name:", seller_name)
else:
    print("Salesperson: Amazon")


    


Seller Name: SUPERBE BEBE®


scrape seller name for list of asins

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup

def get_seller_names(asins):
    # Set up Chrome WebDriver with Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)

    seller_names = {}

    for asin in asins:
        # Navigate to the desired Amazon product page
        product_url = f"https://www.amazon.it/dp/{asin}"  # Use string formatting to replace {asin}
        driver.get(product_url)

        # Extract the page source after the page is loaded
        html = driver.page_source

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find the seller name from the Buy Box
        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        if seller_element is not None:
            seller_name = seller_element.get_text(strip=True)
            seller_names[asin] = seller_name
        else:
            seller_names[asin] = "Amazon"

    # Close the Selenium WebDriver
    driver.quit()

    return seller_names

# Example usage
asins = ['B0B9HS2YCW', 'B072J2J6GZ', 'B07DM7DWJF']
result = get_seller_names(asins)

# Print the seller names
for asin, seller_name in result.items():
    print(f"ASIN: {asin}, Seller Name: {seller_name}")


ASIN: B0B9HS2YCW, Seller Name: SUPERBE BEBE®
ASIN: B072J2J6GZ, Seller Name: Amazon
ASIN: B07DM7DWJF, Seller Name: ALENYK


scrape sellername for asin read from csv and output results in csv


In [ ]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

def get_seller_names(asins):
    # Set up Chrome WebDriver with Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)

    seller_names = {}

    for asin in asins:
        # Navigate to the desired Amazon product page
        product_url = f"https://www.amazon.it/dp/{asin}"  # Use string formatting to replace {asin}
        driver.get(product_url)

        # Extract the page source after the page is loaded
        html = driver.page_source

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find the seller name from the Buy Box
        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        if seller_element is not None:
            seller_name = seller_element.get_text(strip=True)
            seller_names[asin] = seller_name
        else:
            seller_names[asin] = "Amazon"

    # Close the Selenium WebDriver
    driver.quit()

    return seller_names

# Read ASINs from CSV file
def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

# Write seller names to CSV file
def write_seller_names_to_csv(filename, seller_names):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name'])
        for asin, seller_name in seller_names.items():
            csv_writer.writerow([asin, seller_name])

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa Growers/buyboxMonitor/uniqueInputs.csv'
output_csv = '/Users/asifa/Desktop/Asifa Growers/buyboxMonitor/outputnew.csv'
asin_column_name = 'ASIN'  # Specify the column name for ASINs in the CSV

asins = read_asins_from_csv(input_csv)
seller_names = get_seller_names(asins)
write_seller_names_to_csv(output_csv, seller_names)


scrape sellername for asin also the availabilty ,and page not found (final code)

In [10]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

def get_seller_names_and_availability(asins):
    # Set up Chrome WebDriver with Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run Chrome in headless mode
    driver = webdriver.Chrome(options=options)

    seller_data = []

    for asin in asins:
        # Navigate to the desired Amazon product page
        product_url = f"https://www.amazon.it/dp/{asin}"  # Use string formatting to replace {asin}
        driver.get(product_url)

        # Extract the page source after the page is loaded
        html = driver.page_source

        # Parse the page source using BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Find the seller name from the Buy Box
        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        

        # Find the availability <div> element
        availability_div = soup.find('div', {'id': 'availability'})

        if availability_div is not None:
            # Check if the product is available or not
            if "Non disponibile" in availability_div.get_text():
                availability_status = "Not Available"
                seller_name ="none"
            else:
                availability_status = "Available"
                if seller_element is not None:
                    seller_name = seller_element.get_text(strip=True)
                else:
                    seller_name = "Amazon"





        else:
            availability_status = "Product page not found"
            seller_name="none"

        seller_data.append([asin, seller_name, availability_status])

    # Close the Selenium WebDriver
    driver.quit()

    return seller_data

# Read ASINs from CSV file
def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

# Write seller names and availability to CSV file
def write_seller_names_and_availability_to_csv(filename, seller_data):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name', 'Availability_of_Product'])
        for data in seller_data:
            csv_writer.writerow(data)

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/ASIN +Watt 21-07.csv'
output_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/ASIN +Watt 21-07Output.csv'
asin_column_name = 'ASIN'  # Specify the column name for ASINs in the CSV

asins = read_asins_from_csv(input_csv)
seller_data = get_seller_names_and_availability(asins)
write_seller_names_and_availability_to_csv(output_csv, seller_data)


scrapping  sellername for asin also the availabilty ,and page not found and the number of resellers (final code)


In [4]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup


#functin for getting the number of resellers
def get_other_sellers_and_name(asin):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    offerlist_url = f"https://www.amazon.it/dp/{asin}/ref=olp-opf-redir?aod=1&ie=UTF8&condition=NEW&th=1"
    driver.get(offerlist_url)

    html = driver.page_source

    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')

    input_element = soup.find('input', {'id': 'aod-total-offer-count'})
    if input_element is not None:
        num_other_sellers = int(input_element.get('value', '0'))
        
    else:
        num_other_sellers = 0





    seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})
    if seller_element is not None:
        seller_name = seller_element.get_text(strip=True)
    else:
        seller_name = "Amazon"

    return num_other_sellers, seller_name

def get_seller_names_and_availability(asins):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    seller_data = []

    for asin in asins:
        product_url = f"https://www.amazon.it/dp/{asin}"
        driver.get(product_url)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        availability_div = soup.find('div', {'id': 'availability'})

        if availability_div is not None:
            if "Non disponibile" in availability_div.get_text():
                availability_status = "Not Available"
                seller_name = "none"
            else:
                availability_status = "Available"
                if seller_element is not None:
                    seller_name = seller_element.get_text(strip=True)
                else:
                    seller_name = "Amazon"
        else:
            availability_status = "Product page not found"
            seller_name = "none"

        num_other_sellers, _ = get_other_sellers_and_name(asin)
        seller_data.append([asin, seller_name, availability_status, num_other_sellers])

    driver.quit()

    return seller_data

def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

def write_seller_names_and_availability_to_csv(filename, seller_data):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name', 'Availability_of_Product', 'Number_of_resellers'])
        for data in seller_data:
            csv_writer.writerow(data)

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/input2.csv'
output_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/Output2.csv'

asins = read_asins_from_csv(input_csv)
seller_data = get_seller_names_and_availability(asins)
write_seller_names_and_availability_to_csv(output_csv, seller_data)


it gives seller name,reseller name,reseller pricce,number of resellers , only problem: price indexes are wrong


In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

def get_num_of_reseller_and_names(asin):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    offerlist_url = f"https://www.amazon.it/dp/{asin}/ref=olp-opf-redir?aod=1&ie=UTF8&condition=NEW&th=1"
    driver.get(offerlist_url)

    html = driver.page_source

    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')

    input_element = soup.find('input', {'id': 'aod-total-offer-count'})
    if input_element is not None:
        num_other_sellers = int(input_element.get('value', '0'))
    else:
        num_other_sellers = 0

    seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})
    if seller_element is not None:
        seller_name = seller_element.get_text(strip=True)
    else:
        seller_name = "Amazon"

    # Find all the reseller names and prices on the page
    offer_list_div = soup.find('div', {'id': 'aod-offer-list'})
    if offer_list_div:
        reseller_data = []
        reseller_elements = offer_list_div.find_all(
            lambda tag: (tag.name == 'a' and tag.get('aria-label') == 'Apre una nuova pagina') or
                        (tag.name == 'span' and tag.get('aria-label') == 'Amazon. Apre una nuova pagina')
        )
        prices = []  # List to store prices
        for reseller in reseller_elements:
            reseller_name = reseller.get_text(strip=True)
            # Find the corresponding price element for the current reseller
            price_element = reseller.find_next('span', {'class': 'a-offscreen'})
            if price_element:
                # If price found, update the list of prices
                prices.append(price_element.get_text(strip=True))
            reseller_data.append((reseller_name, prices[-1]))  # Use the last price from the list

    else:
        reseller_data = []

    return num_other_sellers, seller_name, reseller_data
   
# Example usage
asin = "B081PV3S3K"
num_sellers, reseller_name, resellers = get_num_of_reseller_and_names(asin)

print(f"Number of Other Sellers: {num_sellers}")
print(f"Seller Name: {reseller_name}")
print("Reseller Names and Prices:")
for reseller, price in resellers:
    print(f" - {reseller}: {price}")


Number of Other Sellers: 7
Seller Name: Aloe Store - L'Angolo dell'Aloe
Reseller Names and Prices:
 - Amazon: 43,30 €
 - risparmia_infarmacia: 38,35 €
 - Mypharmaclick: 42,80 €
 - FarmaciaLoreto: 55,00 €
 - risparmia_infarmacia: 49,66 €
 - Farmaenne: 58,00 €
 - Easyfarma | la tua Farmacia Online: 39,48€


modify code for adding two other column resellers_name and their prices 

In [14]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup


#functin for getting the number of resellers
def get_other_sellers_and_name(asin):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    offerlist_url = f"https://www.amazon.it/dp/{asin}/ref=olp-opf-redir?aod=1&ie=UTF8&condition=NEW&th=1"
    driver.get(offerlist_url)

    html = driver.page_source

    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')

    input_element = soup.find('input', {'id': 'aod-total-offer-count'})
    if input_element is not None:
        num_other_sellers = int(input_element.get('value', '0'))
        
    else:
        num_other_sellers = 0


    # Find all the reseller names on the page
    
    offer_list_div = soup.find('div', {'id': 'aod-offer-list'})
    if offer_list_div:
        reseller_data = []
        reseller_elements = offer_list_div.find_all(
            lambda tag: (tag.name == 'a' and tag.get('aria-label') == 'Apre una nuova pagina') or
                        (tag.name == 'span' and tag.get('aria-label') == 'Amazon. Apre una nuova pagina')
        )
        for reseller in reseller_elements:
            reseller_name = reseller.get_text(strip=True)
           
            reseller_data.append((reseller_name))
    else:
        reseller_data = []

    return num_other_sellers, reseller_data


   

   

def get_seller_names_and_availability(asins):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    seller_data = []

    for asin in asins:
        product_url = f"https://www.amazon.it/dp/{asin}"
        driver.get(product_url)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})

        availability_div = soup.find('div', {'id': 'availability'})

        if availability_div is not None:
            if "Non disponibile" in availability_div.get_text():
                availability_status = "Not Available"
                seller_name = "none"
            else:
                availability_status = "Available"
                if seller_element is not None:
                    seller_name = seller_element.get_text(strip=True)
                else:
                    seller_name = "Amazon"
        else:
            availability_status = "Product page not found"
            seller_name = "none"

        num_other_sellers, _ = get_other_sellers_and_name(asin)
        seller_data.append([asin, seller_name, availability_status, num_other_sellers])

    driver.quit()

    return seller_data

def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

def write_seller_names_and_availability_to_csv(filename, seller_data):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name', 'Availability_of_Product', 'Number_of_resellers','Reseller_names'])
        for data in seller_data:
            csv_writer.writerow(data)

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/input2.csv'
output_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/Output2.csv'

asins = read_asins_from_csv(input_csv)
seller_data = get_seller_names_and_availability(asins)
write_seller_names_and_availability_to_csv(output_csv, seller_data)


scraping seller name,reseller numbers,reseller name,availability (final code)

In [19]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup


# Function for getting the number of resellers and their names
def get_other_sellers_and_name(asin):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    offerlist_url = f"https://www.amazon.it/dp/{asin}/ref=olp-opf-redir?aod=1&ie=UTF8&condition=NEW&th=1"
    driver.get(offerlist_url)

    html = driver.page_source

    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')

    input_element = soup.find('input', {'id': 'aod-total-offer-count'})
    if input_element is not None:
        num_other_sellers = int(input_element.get('value', '0'))
    else:
        num_other_sellers = 0

    # Find all the reseller names on the page
    offer_list_div = soup.find('div', {'id': 'aod-offer-list'})
    if offer_list_div:
        reseller_data = []
        reseller_elements = offer_list_div.find_all(
            lambda tag: (tag.name == 'a' and tag.get('aria-label') == 'Apre una nuova pagina') or
                        (tag.name == 'span' and tag.get('aria-label') == 'Amazon. Apre una nuova pagina')
        )
        for reseller in reseller_elements:
            reseller_name = reseller.get_text(strip=True)
            reseller_data.append(reseller_name)
    else:
        reseller_data = 'No Resellers'

    return num_other_sellers, reseller_data


def get_seller_names_and_availability(asins):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    seller_data = []

    for asin in asins:
        product_url = f"https://www.amazon.it/dp/{asin}"
        driver.get(product_url)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})
        availability_div = soup.find('div', {'id': 'availability'})

        if availability_div is not None:
            if "Non disponibile" in availability_div.get_text():
                availability_status = "Not Available"
                seller_name = "none"
            else:
                availability_status = "Available"
                if seller_element is not None:
                    seller_name = seller_element.get_text(strip=True)
                else:
                    seller_name = "Amazon"
        else:
            availability_status = "Product page not found"
            seller_name = "none"

        num_other_sellers, reseller_names = get_other_sellers_and_name(asin)
        seller_data.append([asin, seller_name, availability_status, num_other_sellers, reseller_names])

    driver.quit()

    return seller_data

def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

def write_seller_names_and_availability_to_csv(filename, seller_data):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name', 'Availability_of_Product', 'Number_of_resellers', 'Reseller_names'])
        for data in seller_data:
            csv_writer.writerow(data)

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/input2.csv'
output_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/Output2.csv'

asins = read_asins_from_csv(input_csv)
seller_data = get_seller_names_and_availability(asins)
write_seller_names_and_availability_to_csv(output_csv, seller_data)


modifying the final code for convert array reseller names into list (DONE)

In [2]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup


# Function for getting the number of resellers and their names
def get_other_sellers_and_name(asin):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    offerlist_url = f"https://www.amazon.it/dp/{asin}/ref=olp-opf-redir?aod=1&ie=UTF8&condition=NEW&th=1"
    driver.get(offerlist_url)

    html = driver.page_source

    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')

    input_element = soup.find('input', {'id': 'aod-total-offer-count'})
    if input_element is not None:
        num_other_sellers = int(input_element.get('value', '0'))
    else:
        num_other_sellers = 0

    # Find all the reseller names on the page
    offer_list_div = soup.find('div', {'id': 'aod-offer-list'})
    if offer_list_div:
        reseller_data = []
        reseller_elements = offer_list_div.find_all(
            lambda tag: (tag.name == 'a' and tag.get('aria-label') == 'Apre una nuova pagina') or
                        (tag.name == 'span' and tag.get('aria-label') == 'Amazon. Apre una nuova pagina')
        )
        for reseller in reseller_elements:
            reseller_name = reseller.get_text(strip=True)
            reseller_data.append(reseller_name)
    else:
        reseller_data = 'No Resellers'

    return num_other_sellers, reseller_data


def get_seller_names_and_availability(asins):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    seller_data = []

    for asin in asins:
        product_url = f"https://www.amazon.it/dp/{asin}"
        driver.get(product_url)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})
        availability_div = soup.find('div', {'id': 'availability'})

        if availability_div is not None:
            if "Non disponibile" in availability_div.get_text():
                availability_status = "Not Available"
                seller_name = "none"
            else:
                availability_status = "Available"
                if seller_element is not None:
                    seller_name = seller_element.get_text(strip=True)
                else:
                    seller_name = "Amazon"
        else:
            availability_status = "Product page not found"
            seller_name = "none"

        num_other_sellers, reseller_names = get_other_sellers_and_name(asin)
        if num_other_sellers > 0:      
            resellers_with_numbers = [f"{i+1}. {reseller}\n" for i, reseller in enumerate(reseller_names)]
            seller_data.append([asin, seller_name, availability_status, num_other_sellers, "".join(resellers_with_numbers)])
        else:
            seller_data.append([asin, seller_name, availability_status, num_other_sellers,resellers_with_numbers:='No Reseller'])


    driver.quit()

    return seller_data

def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

def write_seller_names_and_availability_to_csv(filename, seller_data):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name', 'Availability_of_Product', 'Number_of_resellers', 'Reseller_names'])
        for data in seller_data:
            csv_writer.writerow(data)

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/input2.csv'
output_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/Output2.csv'

asins = read_asins_from_csv(input_csv)
seller_data = get_seller_names_and_availability(asins)
write_seller_names_and_availability_to_csv(output_csv, seller_data)


MODIFYING PREVIOUS CODE TO ADD PRICES OF THE RESELERS

In [38]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup


# Function for getting the number of resellers and their names
def get_other_sellers_and_name(asin):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    offerlist_url = f"https://www.amazon.it/dp/{asin}/ref=olp-opf-redir?aod=1&ie=UTF8&condition=NEW&th=1"
    driver.get(offerlist_url)

    html = driver.page_source

    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')

    input_element = soup.find('input', {'id': 'aod-total-offer-count'})
    if input_element is not None:
        num_other_sellers = int(input_element.get('value', '0'))
    else:
        num_other_sellers = 0

    # Find all the reseller names on the page
    offer_list_div = soup.find('div', {'id': 'aod-offer-list'})
    if offer_list_div:
        reseller_data = []
        reseller_elements = offer_list_div.find_all(
            lambda tag: (tag.name == 'a' and tag.get('aria-label') == 'Apre una nuova pagina') or
                        (tag.name == 'span' and tag.get('aria-label') == 'Amazon. Apre una nuova pagina')
        )
        for reseller in reseller_elements:
            reseller_name = reseller.get_text(strip=True)
            reseller_data.append(reseller_name)
    else:
        reseller_data = 'No Resellers'
    if offer_list_div:
        reseller_price_data = []
        # Find all the spans containing prices with class 'a-offscreen' and 'a-price'
        price_spans = offer_list_div.find_all('span', class_=['a-price'])
        reseller_price_data = [price_span.text.strip() for price_span in price_spans]
        
    else:
        reseller_price_data = 'No Prices'
    

    

    return num_other_sellers, reseller_data, reseller_price_data


def get_seller_names_and_availability(asins):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    seller_data = []

    for asin in asins:
        product_url = f"https://www.amazon.it/dp/{asin}"
        driver.get(product_url)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        seller_element = soup.find('a', {'id': 'sellerProfileTriggerId'})
        availability_div = soup.find('div', {'id': 'availability'})

        if availability_div is not None:
            if "Non disponibile" in availability_div.get_text():
                availability_status = "Not Available"
                seller_name = "none"
            else:
                availability_status = "Available"
                if seller_element is not None:
                    seller_name = seller_element.get_text(strip=True)
                else:
                    seller_name = "Amazon"
        else:
            availability_status = "Product page not found"
            seller_name = "none"

        num_other_sellers, reseller_names, reseller_price_data = get_other_sellers_and_name(asin)
        if num_other_sellers > 0:
            resellers_with_numbers = [f"{i+1}. {reseller} - Price: {price}\n" for i, (reseller, price) in enumerate(zip(reseller_names, reseller_price_data))]
            seller_data.append([asin, seller_name, availability_status, num_other_sellers, "".join(resellers_with_numbers)])
        else:
            seller_data.append([asin, seller_name, availability_status, num_other_sellers, 'No Reseller', 'No Prices'])

    driver.quit()

    return seller_data

def read_asins_from_csv(filename):
    asins = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)
        for row in csv_reader:
            asins.extend(row)
    return asins

def write_seller_names_and_availability_to_csv(filename, seller_data):
    with open(filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['ASIN', 'Seller Name', 'Availability_of_Product', 'Number_of_resellers', 'Reseller_names','reseller_price'])
        for data in seller_data:
            csv_writer.writerow(data)

# Example usage
input_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/input2.csv'
output_csv = '/Users/asifa/Desktop/Asifa/buyboxMonitor/Output2.csv'

asins = read_asins_from_csv(input_csv)
seller_data = get_seller_names_and_availability(asins)
write_seller_names_and_availability_to_csv(output_csv, seller_data)
